In [ ]:
!pip install -r /content/requirements.txt -q

In [ ]:
!pip install pypdf -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 277.9/277.9 kB 4.8 MB/s eta 0:00:00


In [ ]:
!pip install langchain -q

In [ ]:
import os
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import WikipediaLoader


In [ ]:
from dotenv import load_dotenv,find_dotenv
load_dotenv('/content/drive/MyDrive/Environments/.env',override=True)


True

In [ ]:
!pip install docx2txt -q

  Preparing metadata (setup.py) ... done


In [ ]:
!pip install wikipedia -q

  Preparing metadata (setup.py) ... done


In [ ]:
def load_pdf_document(file):
  print(f'Loading {file}')
  loader=PyPDFLoader(file)
  data = loader.load()
  return data

In [ ]:
#upload any file format

def load_document(file):
  import os
  name,extension = os.path.splitext(file)
  if extension =='.pdf':
    data = load_pdf_document(file)
  elif extension =='.docx':
    from langchain.document_loaders import Docx2txtLoader
    print(f'Loading {file}...')
    loader = Docx2txtLoader(file)
  else:
    print('Document format is not supported')

  data = loader.load()
  return data


In [ ]:
def load_from_wikipedia(query , lang='en'):
  loader = WikipediaLoader(query=query, lang=lang, load_max_docs = 2) #limits the number fo dowloaded pages.
  data=loader.load()
  return data

In [ ]:
def chunk_data(data, chunk_size = 256):
  from langchain.text_splitter import RecursiveCharacterTextSplitter
  text_splitter = RecursiveCharacterTextSplitter(chunk_size = chunk_size, chunk_overlap=0)
  chunks= text_splitter.split_documents(data) #use split_documents when documents are already splitted by page
  return chunks


In [ ]:
import tiktoken
def print_embedding_cost(texts):
  enc = tiktoken.encoding_for_model('text-embedding-ada-002')
  total_tokens = sum([len(enc.encode(page.page_content)) for page in texts])
  print(f'Total Tokens : {total_tokens}')
  print(f'Embedding Cost in USD: {total_tokens/1000*0.0004:.6f}')

In [ ]:
def insert_or_fetch_embeddings(index_name):
  import pinecone
  from langchain.vectorstores import Pinecone #Pinecone class
  from langchain.embeddings.openai import OpenAIEmbeddings

  embeddings = OpenAIEmbeddings()
  pinecone.init(api_key = os.environ.get('PINECONE_API_KEY'), environment=os.environ.get('PINECONE_env'))

  if index_name in pinecone.list_indexes():
    vector_store = Pinecone.from_existing_index(index_name, embeddings)
  else:
    print(f'Creating index {index_name} and embeddings...', end='')
    pinecone.create_index(index_name, dimension =1536, metric = 'cosine')
    vector_store=Pinecone.from_documents(chunks, embeddings, index_name=index_name)
    print('Done!')
  return vector_store

In [ ]:
def del_pinecone_index(index_name='all'):
  import pinecone
  pinecone.init(api_key = os.environ.get('PINECONE_API_KEY'), environment=os.environ.get('PINECONE_env'))
  if index_name=='all':
      indexes= pinecone.list_indexes()
      print('Deleting all indexes...')
      for index in indexes:
        pinecone.delete_index(index)
      print('Ok')
  else:
    print(f'Deleting {index_name}...' , end='')
    pinecone.delete_index(index_name)
    print('Ok')



In [ ]:
def ask_with_memory(vector_store , question , chat_history = []):
  from langchain.chains import ConversationalRetrievalChain
  from langchain.chat_models  import ChatOpenAI

  llm = ChatOpenAI(temperature = 1)
  retriever = vector_store.as_retriever(search_type = 'similarity', search_kwargs = ({'k':3}))
  crc = ConversationalRetrievalChain.from_llm(llm , retriever)
  result = crc({'question':question , 'chat_history':chat_history})
  chat_history.append((question , result['answer']))

  return result , chat_history


## Running the file


In [ ]:
data_mobile = load_pdf_document('/content/Lecture 2 - History of Mobile Robots - W23.pdf')
print(data_mobile[1].page_content)

Loading /content/Lecture 2 - History of Mobile Robots - W23.pdf
Lecture 2:  History of Mobile Robots 2800 BCE | Golden Tripods
Thetis' silver feet took her to Hephaestus' house, 
A mansion the lame god had built himself... 
She found him at his bellows, glazed with sweat 
As he hurried to complete his latest project, 
Twenty cauldrons on tripods to line his hall 
With golden wheels at the base of each tripod. 
....He was getting these ready, 
Forging the rivets with inspired artistry.
-Homer, Iliad, 18:398, 8th century BC (Lombardo translation)           
first concept for a mobile, multi -
robot, autonomous, wheeled, 
delivery application robot team
•Hephaestus is the Greek 
god of blacksmiths and 
metalworking. In the 
Illiad , he creates wheeled 
tripods to help him.
Mount Olympus depiction on 
early pottery
 Tripods


In [ ]:
print(data[10].page_content)

Lecture 2:  History of Mobile RobotsProbabilistic Computationalism |Bayesian Inference
Thomas BayesProbabilities as Gaussian PDFs
Probabilities as Sample Sets•Principled fusion of probability 
distributions from multiple 
sources
•Priors over states
•Probabilistic models for 
measurements
•Probabilistic models for 
evolution of the robot state 
and environment


In [ ]:
print(f'The document has {len(data)} pages')

The document has 49 pages


In [ ]:
data = load_from_wikipedia('GPT-4')
print(data[0].page_content)

Generative Pre-trained Transformer 4 (GPT-4) is a multimodal large language model created by OpenAI, and the fourth in its series of GPT foundation models. It was initially released on March 14, 2023, and has been made publicly available via the paid chatbot product ChatGPT Plus, and via OpenAI's API.  As a transformer-based model, GPT-4 uses a paradigm where pre-training using both public data and "data licensed from third-party providers" is used to predict the next token. After this step, the model was then fine-tuned with reinforcement learning feedback from humans and AI for human alignment and policy compliance.: 2 Observers reported that the iteration of ChatGPT using GPT-4 was an improvement on the previous iteration based on GPT-3.5, with the caveat that GPT-4 retains some of the problems with earlier revisions. GPT-4 is also capable of taking images as input on ChatGPT. OpenAI has declined to reveal various technical details and statistics about GPT-4, such as the precise siz

In [ ]:
# Chunking strategies -  a chunk of text should make sense without the other content

chunks = chunk_data(data_mobile)
print(f'We have {len(chunks)} chunks')


We have 124 chunks


In [ ]:
print(chunks[3].page_content)

Twenty cauldrons on tripods to line his hall 
With golden wheels at the base of each tripod. 
....He was getting these ready, 
Forging the rivets with inspired artistry.
-Homer, Iliad, 18:398, 8th century BC (Lombardo translation)


In [ ]:
print_embedding_cost(chunks)

Total Tokens : 6094
Embedding Cost in USD: 0.002438


In [ ]:
#embedding and uplaoding chunks to pinecone databse
del_pinecone_index()

Deleting all indexes...


In [ ]:
index_name = 'askaquestion'
vector_store = insert_or_fetch_embeddings(index_name)

Creating index askaquestion and embeddings...Done!


In [ ]:
#Query the database

def QnA(vector_store , q):
  from langchain.chains import RetrievalQA
  from langchain.chat_models import ChatOpenAI

  llm=ChatOpenAI(model='gpt-3.5-turbo', temperature = 1)
  retriever = vector_store.as_retriever(search_type = 'similarity', search_kwargs = ({'k':3}))
  chain = RetrievalQA.from_chain_type(llm=llm, chain_type='stuff',retriever = retriever)
  answer = chain.run(q)
  return answer

In [ ]:
q = 'What is the whole document about?'
answer = QnA(vector_store , q)
print(answer)

The document is likely about the history and advancements in the field of mobile robots, including specific examples such as the Braitenberg Vehicles and the DARPA Urban Grand Challenge. It may also discuss different approaches to mobile robot behavior, such as finite state machines.


In [ ]:
import time
i=1
print('Write Quit or Exit to quit.')
while True:
  q=input(f'Question #{i}: ')
  i=i+1
  if q.lower() in ['quit' , 'exit']:
    print('Bye Bye!')
    time.sleep(2)
    break
  answer = QnA(vector_store,q)
  print(f'\nAnwer: {answer}')
  print(f'\n{"-"*50} \n')

Write Quit or Exit to quit.
Question #1: what is the document about?

Anwer: The document appears to be about the history of mobile robots, specifically mentioning the DARPA Urban Grand Challenge in 2007 and the Braitenberg Vehicles. It also discusses various milestones and achievements in the field of mobile robotics, such as the first outdoor mobile robot and the first mobile robot on another planetary surface.

-------------------------------------------------- 

Question #2: what are Braitenberg Vehicles?

Anwer: Braitenberg Vehicles are a concept introduced by Valentin von Braitenberg in his book "Vehicles: Experiments in Synthetic Psychology". They are hypothetical vehicles or machines that exhibit simple behaviors based on their sensor and motor connections. Braitenberg describes 14 different vehicles in his book, each with increasingly complex layers of behavior. These vehicles demonstrate various phenomena such as foresight, intentionality, individuation, egoism, and optimism.

In [ ]:
def QnA_w_history(vector_store , q, chat_history =[]):
  from langchain.chains import ConversationalRetrievalChain
  from langchain.chat_models import ChatOpenAI

  llm=ChatOpenAI(model='gpt-3.5-turbo', temperature = 1)
  retriever = vector_store.as_retriever(search_type = 'similarity', search_kwargs = ({'k':3}))
  chain_conv = ConversationalRetrievalChain.from_llm(llm=llm,retriever = retriever)
  result = chain_conv({'question': q , 'chat_history': chat_history})
  chat_history.append((q , result['answer']))
  return result , chat_history

In [ ]:
chat_history=[]
q='What are mobile robotics'
result = QnA_w_history(vector_store , q, chat_history= [])
print(result['answer'])
print(chat_history)

TypeError: ignored

In [ ]:
#asking with chat history
chat_history = []
question =  'What are mobile robotics?'
result , chat_history = ask_with_memory(vector_store , question , chat_history = [])
print(result['answer'])
print(chat_history)

In [ ]:
question2 = '....'
result , chat_history = ask_with_memory(vector_store , question2 , chat_history = [])
print(result['answer'])
print(chat_history)